In [31]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

from api_keys import g_key

In [32]:
# Store Part I results into DataFrame

cities_df = pd.read_csv('output_data/city_weather.csv')
cities_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Mar del Plata,-38.0023,-57.5575,47.37,73,0,7.76,AR,1632266546
1,1,Woodward,36.4337,-99.3904,74.97,30,1,13.80,US,1632266547
2,2,Thompson,55.7435,-97.8558,48.36,61,40,2.30,CA,1632266547
3,3,Souillac,-20.5167,57.5167,70.18,64,75,11.50,MU,1632266248
4,4,Kirakira,-10.4544,161.9205,83.70,72,48,5.48,SB,1632266548


In [33]:
# Humidity Heatmap

# Configure gmaps
# Use the Lat and Lng as locations and Humidity as the weight
# Add Heatmap layer to map

gmaps.configure(api_key = g_key)

In [34]:
locations = cities_df[["Lat", "Lng"]]
humidity = cities_df["Humidity"].astype(float)
maxhumidity = humidity.max()

heatmap_fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(
    locations, weights = humidity, 
    dissipating = False, max_intensity = maxhumidity,
    point_radius = 3
    )

heatmap_fig.add_layer(heat_layer)
heatmap_fig

Figure(layout=FigureLayout(height='420px'))

In [54]:
# Create new DataFrame fitting weather criteria

# Narrow down the cities to fit weather conditions
# Drop any rows will null values

narrowed_city_df = cities_df.loc[(cities_df["Wind Speed"] < 10) & (cities_df["Max Temp"] > 70) & (cities_df["Max Temp"] < 80) & (cities_df["Cloudiness"] == 0), :]
narrowed_city_df = narrowed_city_df.dropna(how = 'any')
narrowed_city_df.reset_index(inplace = True)
del narrowed_city_df['index']
narrowed_city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,10,Conde,-7.2597,-34.9075,76.46,88,0,5.75,BR,1632266400
1,21,Balkanabat,39.5108,54.3671,75.42,17,0,7.90,TM,1632266554
2,77,Mitsamiouli,-11.3847,43.2844,75.81,73,0,4.45,KM,1632266575
3,119,Nanzhou,29.3596,112.4024,70.83,81,0,6.76,CN,1632266592
4,199,Dalbandin,28.8947,64.4101,75.54,14,0,1.25,PK,1632266626


In [55]:
# Hotel Map

hotellist = []

for i in range(len(narrowed_city_df)):
    lat = narrowed_city_df.loc[i]['Lat']
    lng = narrowed_city_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
        
narrowed_city_df["Hotel Name"] = hotellist
narrowed_city_df = narrowed_city_df.dropna(how='any')
narrowed_city_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
0,10,Conde,-7.2597,-34.9075,76.46,88,0,5.75,BR,1632266400,João Pessoa
1,21,Balkanabat,39.5108,54.3671,75.42,17,0,7.90,TM,1632266554,Balkanabat
2,77,Mitsamiouli,-11.3847,43.2844,75.81,73,0,4.45,KM,1632266575,Mitsamiouli
3,119,Nanzhou,29.3596,112.4024,70.83,81,0,6.76,CN,1632266592,Yueyang
4,199,Dalbandin,28.8947,64.4101,75.54,14,0,1.25,PK,1632266626,Dalbandin


In [56]:
# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row

hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = narrowed_city_df[["Lat", "Lng"]]

In [57]:
markers = gmaps.marker_layer(locations)
heatmap_fig.add_layer(markers)
heatmap_fig

Figure(layout=FigureLayout(height='420px'))